# Explainability notebook

In [ ]:
# Updated to use new modular structure
import os
import pandas as pd
from src.data import load_dataset
from src.cleaning import quick_clean
from src.features import safe_feature_columns
from src.prediction import load_predictor
from src.explainability import compute_shap_values, plot_shap_summary

# Load model
print("Loading model...")
predictor = load_predictor(model_name="best_classifier_mortality")

# Load and prepare data
dataset_path = os.environ.get('DATASET_PATH', '../DATA/recuima-020425-fragment.csv')
df = load_dataset(dataset_path)
df_clean = quick_clean(df, target_column="mortalidad")

# Select features
feature_cols = safe_feature_columns(df_clean, exclude_cols=["mortalidad"])
X = df_clean[feature_cols].iloc[:200]  # Use first 200 samples for speed

print(f"Computing SHAP values for {X.shape[0]} samples...")

# Compute SHAP values
try:
    explainer, shap_values = compute_shap_values(
        predictor.model, 
        X.values, 
        feature_names=feature_cols,
        max_samples=100
    )
    
    # Plot summary
    path = plot_shap_summary(
        shap_values, 
        X.values, 
        feature_names=feature_cols,
        name='mortality',
        max_display=20
    )
    
    print(f"SHAP summary plot saved to: {path}")
except Exception as e:
    print(f"Error computing SHAP values: {e}")
    print("Make sure SHAP is installed: pip install shap")
